In [0]:
'''IMPORTING PYTHON LIBRARIES'''
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [0]:
'''IMPORTING TENSORFLOW LIBRARIES'''
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, UpSampling2D, Concatenate
from tensorflow.python.keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D,merge, Add
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import regularizers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [0]:
'''LOADING GOOGLE DRIVE'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
'''READING THE DATASET'''



train_labels = pickle.load(open("/content/drive/My Drive/ML_project/Lane_Detection/full_CNN_labels.p", "rb" ))
train_images = pickle.load(open("/content/drive/My Drive/ML_project/Lane_Detection/full_CNN_train.p", "rb" ))
train_images = np.array(train_images)
train_labels = np.array(train_labels)/255

In [0]:
def conv_block(X,kernal_size,num_of_filter,stride):

  '''
  This function defines the convolution block used in the model.
  First layer is a 1x1 colvolution layer.
  Second layer is a batch normalization layer.
  third layer is (kernal_size x kernal_size) convolution layer
  forth layer is batch normalization
  fifth layer is (kernal_size x kernal_size) convolution layer
  sixth layer is batch normalization

  We call this multiple number of times to implement UNET structure
  '''


  out_put = Conv2D(num_of_filter[0], strides = (stride,stride), kernel_size=(1,1), padding='same', activation='relu', kernel_initializer = tf.keras.initializers.he_uniform())(X)
  out_put = BatchNormalization(axis=3)(out_put)
  #out_put = Dropout(0.2)(out_put)
  
  out_put = Conv2D(num_of_filter[1], strides = (stride,stride), kernel_size=(kernal_size,kernal_size), padding='same', activation='relu', kernel_initializer = tf.keras.initializers.he_uniform())(out_put)
  out_put = BatchNormalization(axis=3)(out_put)
  #out_put = Dropout(0.2)(out_put)
  
  out_put = Conv2D(num_of_filter[2], strides = (stride,stride), kernel_size=(kernal_size,kernal_size), padding='same', activation='relu', kernel_initializer = tf.keras.initializers.he_uniform())(out_put)
  out_put = BatchNormalization(axis=3)(out_put)
  #out_put = Dropout(0.2)(out_put)

  
  return out_put

In [0]:
def De_conv_block(X,kernal_size,num_of_filter):

  '''
  In this function, we try to implement the transpose convolution layer.
  '''

  x = UpSampling2D((2, 2))(X)
  y1 = Conv2DTranspose(num_of_filter, (kernal_size, kernal_size), activation='relu', padding='same')(x) 

  return y1

In [0]:
'''PRE-PROCESSING'''


train_images, train_labels = shuffle(train_images, train_labels)
# Test size may be 10% or 20%
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2)

# Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
batch_size = 128
epochs = 10
pool_size = (2, 2)
input_shape = X_train.shape[1:]

In [0]:

'''MODEL DEFINiTION'''


tf.compat.v1.reset_default_graph()

X_input = tf.keras.Input(input_shape)
'''Block 1 Convolution'''
down1 = conv_block(X_input,3,[32,32,32],1)
pool = MaxPooling2D((2, 2), padding='same')(down1)

'''Block 2 Convolution'''
down2 = conv_block(pool,3,[64,64,64],1)
pool = MaxPooling2D((2, 2), padding='same')(down2)

'''Block 3 Convolution'''
down3 = conv_block(pool,3,[128,128,128],1)
pool = MaxPooling2D((2, 2), padding='same')(down3)

'''Block 4 Convolution'''
down4 = conv_block(pool,3,[256,256,256],1)
pool = MaxPooling2D((2, 2), padding='same')(down4)

'''Equilizer'''
eq = Conv2D(512, strides = (1,1), kernel_size=(1,1), padding='same', activation='relu', kernel_initializer = tf.keras.initializers.he_uniform())(pool)


'''Block 1 Deconvolve'''
up1 = De_conv_block(eq,3,256)
up1 = conv_block(up1,3,[256,256,256],1)
up1 = Add()([up1,down4])

'''Block 1 Deconvolve'''
up2 = De_conv_block(up1,3,128)
up2 = conv_block(up2,3,[128,128,128],1)
up2 = Add()([up2,down3])

'''Block 1 Deconvolve'''
up3 = De_conv_block(up2,3,64)
up3 = conv_block(up3,3,[64,64,64],1)
up3 = Add()([up3,down2])

'''Block 1 Deconvolve'''
up4 = De_conv_block(up3,3,32)
up4 = conv_block(up4,3,[32,32,32],1)
up4 = Add()([up4,down1])

out1 = Conv2D(16, strides = (1,1), kernel_size=(3,3), padding='same', activation='relu', kernel_initializer = tf.keras.initializers.he_uniform())(up4)
out2 = Conv2D(8, strides = (1,1), kernel_size=(3,3), padding='same', activation='relu', kernel_initializer = tf.keras.initializers.he_uniform())(out1)
out3 = Conv2D(1, strides = (1,1), kernel_size=(3,3), padding='same', activation='sigmoid', kernel_initializer = tf.keras.initializers.he_uniform())(out2)

In [0]:
'''MODEL COMBINITION'''

encoder = tf.keras.Model(X_input, out3, name='encoder')
encoder.summary()



Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 160, 3)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 80, 160, 32)  128         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 80, 160, 32)  128         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 80, 160, 32)  9248        batch_normalization[0][0]        
____________________________________________________________________________________________

In [0]:
'''Data Augmentation'''


datagen = ImageDataGenerator(channel_shift_range=0.2)
datagen.fit(X_train)


In [0]:
'''Compiling and training the model'''

encoder.compile(optimizer='Adam', loss='mean_squared_error')
history = encoder.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,epochs=epochs, verbose=1, validation_data=(X_val, y_val))

In [0]:
'''Saving the model'''

encoder.save('/content/drive/My Drive/ML_project/Lane_Detection/final_model.h5')